In [40]:
%%bash 

# first have to rename the goddamn sequencing reads 

for i in fwd_reads/*seq rev_reads/*seq; do
    cat $i > $( echo $i | sed -e 's/\([A-Z]\)\([A-Z]\)/\1+\2/g' ) && rm $i
done

In [2]:
!head -2 fwd_reads/*

==> fwd_reads/A100F+V105A+Y168A+K172Y-1_T7_Plate_Plate02_H05.seq <==
>A100FV105AY168AK172Y-1_T7 Sample_Name=A100FV105AY168AK172Y-1_T7 Chromat_id=4467614 Read_id=4416595 Version=1 Length=1176
NNNNNNNNNNNNNNNNNNNCTAGNATANTTTTGTTTAACTTTAAGAAGGAGATATACATATGGCAGATGGTCTGGGTTGTGCAGTTTGTGTTCTGACCGGTGCAAGCCGTGGTTTTGGTCGTGCACTGGCACCGCAGCTGGCACGTCTGCTGAGTCCGGGTAGCGTTATGCTGGTTAGCGCACGTAGCGAAAGCATGCTGCGTCAGCTGAAAGAAGAACTGGGAGCACAGCAGCCGGATCTGAAAGTTGTTCTGGCAGCAGCAGATCTGGGCACCGAAGCCGGTGTTCAGCGTCTGCTGTCAGCAGTTCGTGAACTGCCTCGTCCGGAAGGTCTGCAGCGCCTGCTGCTGATTAATAACGCATTTACCCTGGGTGATGCGAGCAAAGGTTTTCTGAATGTTAATGATCTGGCCGAGGTGAATAACTATTGGGCACTGAATCTGACCAGTATGCTGTGTCTGACCAGCGGCACCCTGAATGCCTTTCAGGATAGTCCGGGTCTGAGCAAAACCGTTGTGAATATTAGCAGCCTGTGTGCACTGCAGCCGTATAAAGGTTGGGGTCTGGCGTGTGCAGGTTATGCAGCACGTGATATGCTGTATCAGGTGCTGGCAGCCGAAGAACCGAGCGTTCGTGTTCTGAGCTATGCACCGGGTCCGCTGGATAATGATATGCAGCAACTGGCACGTGAAACCAGCAAAGATCCGGAACTGCGTAGCAAACTGCAGAAACTGAAAAGTGATGGTGCACTGGTTGATTGTGGCACCAGCGCACAGAAACTGCTGGGTCTGCTGCAGAAAGATACCTTT

In [3]:
%%bash 

FN=kunkelWellCSV_r18cdhv8exmh3.csv

dos2unix $FN # fix any stupid DOS input files 
mkdir out 

for i in $( cut -d, -f3 $FN ); do
  ASEQ=fwd_reads/$i*seq
  BSEQ=rev_reads/$i*seq
  merger -asequence $ASEQ -bsequence $BSEQ -sreverse2 -outfile out/$i.merger -outseq out/$i.fasta
done

dos2unix: converting file kunkelWellCSV_r18cdhv8exmh3.csv to Unix format...
mkdir: out: File exists
Merge two overlapping sequences
Error: Failed to open filename 'fwd_reads/mutant*seq'
Error: Unable to read sequence 'fwd_reads/mutant*seq'
Died: merger terminated: Bad value for '-asequence' and no prompt
Merge two overlapping sequences
Error: Failed to open filename 'fwd_reads/name*seq'
Error: Unable to read sequence 'fwd_reads/name*seq'
Died: merger terminated: Bad value for '-asequence' and no prompt
Merge two overlapping sequences
Merge two overlapping sequences
Merge two overlapping sequences
Merge two overlapping sequences
Merge two overlapping sequences
Merge two overlapping sequences
Merge two overlapping sequences
Merge two overlapping sequences
Merge two overlapping sequences
Merge two overlapping sequences
Merge two overlapping sequences
Merge two overlapping sequences
Merge two overlapping sequences
Merge two overlapping sequences
Merge two overlapping sequences
Merge two ov

In [12]:
%%bash 

cat out/*fasta > reads.fasta 
lastdb -c designs_db reads.fasta
lastal designs_db 1oaa.fasta

# LAST version 581
#
# a=7 b=1 A=7 B=1 e=23 d=14 x=22 y=9 z=22 D=1e+06 E=6.22339e+06
# R=10 u=2 s=2 T=0 m=10 l=1 n=10 k=1 i=8192 w=1000 t=0.910239 g=1 j=3 Q=0
# designs_db
# Reference sequences=60 normal letters=80342
# lambda=1.09602 K=0.335388
#
#    A  C  G  T
# A  1 -1 -1 -1
# C -1  1 -1 -1
# G -1 -1  1 -1
# T -1 -1 -1  1
#
# Coordinates are 0-based.  For - strand matches, coordinates
# in the reverse complement of the 2nd sequence are used.
#
# name start alnSize strand seqSize alignment
#
# batch 0
a score=728 EG2=0 E=0
s L102AV105HS155AY168HA195CM203W-2_T7 398 805 + 1401 gcagatggtctgggtTgtgcagtttgtgttctgaccggtgcaagccgtggttttggtcgtgcactggcaccgcagctggcacgtctgctgagtccgggtagcgttatgctggttagcgcacgtagcgaaagcatgctgcgtcagctgaaagaagaactgggagcacagcagccggatctgaaagttgttctggcagcagcagatctgggcaccgaagccggtgttcagcgtctgctgtcagcagttcgtgaactgcctcgtccggaaggtctgcagcgcctgctgctgattaataacgcagcaaccgcgggtgatcatagcaaaggttttctgaatgttaatgatctggccgaggtgaataactattgggcactgaatctgaccagtatgctgtgtctgaccagcggcaccctga

In [7]:
from subprocess import call 
from glob import glob 
import pandas 

well_map = 'kunkelWellCSV_r18cdhv8exmh3.csv'

# diff and decide 
df = pandas.read_csv( well_map )
df.set_index( 'mutant name', inplace=True )

picks = { }
for fil in glob( 'out/*diff' ):
    #print fil
    with open( fil ) as fn:
        lines = fn.readlines()
        if len( lines ) > 0:
            #print len( lines )
            #print lines[0]
            sub, que = lines[0].split( '\t' )   
            d = '+'.join( [ '{}{}{}'.format(i, n, j).lower() for (n, (i, j)) in enumerate( zip( sub, que ) ) if i != j ] ) 
            mutant_label = fil.split( '/' )[1].split( '.' )[0].lower()
            mutant_name = mutant_label.split( '-' )[0]
            if mutant_name == d:
                #print 'found one!'
                #print mutant_label, mutant_name, d
                picks.update( { mutant_name: df.loc[ mutant_label.upper() ].well } )

print pandas.Series( picks )
print len( picks )
#clones = pandas.DataFrame( picks, columns=['clone', 'well'] )
#clones.to_csv( 'clones.csv' )

#print "Diagnostics:"
#print 'Found {} of {} clones are correct ({}%)'.format( len( clones ), len( df ), 100 * len( clones ) / len( df ) )
#uniqs = set( [ m.split('-')[0] for m in df.index ] )
#uniq_clones = set( [ m.split('-')[0].upper() for m in clones.clone ] )
#print uniqs
#print uniq_clones
#print 'Found {} of {} ordered mutants (missing {})'.format( len( uniq_clones ), len( uniqs ), len( uniqs.difference( uniq_clones ) ) )

#print 'Original order:'
#for i in uniqs:
#    print i 
#print 'Not found:'
#for i in uniqs.difference( uniq_clones ):
#    print i

a100f+v105a+y168a+k172y                      B6
a100s+l102h+v105a+y168a+k172y                G5
a100s+l102k+v105a+y168a+k172y                C1
a100s+y168a+k172y                            A5
a100y+l102k+v105a+y168a+k172y                B3
l102a+c157s+y168h+k172s+m203w                F4
l102a+c157s+y168h+m203w                      A4
l102a+c157w+y168h+m203w                      F1
l102a+s155q+y168h+m203w                      C7
l102a+v105h+s155a+y168h+a195c+m203w          E6
l102a+v105h+s155a+y168h+a195c+m203w+a207m    D3
l102a+v105h+s155a+y168h+m203w                C2
l102a+v105h+s155q+y168h+m203w                G3
l102a+y168h+m203w                            A8
l102g+v105g+y168a+k172y+n201d+q204a+a207h    G2
y168a+k172y+n201d+q204a                      C5
dtype: object
16


In [62]:
!cat clones.csv

,clone,well
0,a100f+v105a+y168a+k172y-1,H5
1,a100f+v105a+y168a+k172y-2,A6
2,a100f+v105a+y168a+k172y-3,B6
3,a100s+l102h+v105a+y168a+k172y-1,E5
4,a100s+l102h+v105a+y168a+k172y-2,F5
5,a100s+l102h+v105a+y168a+k172y-3,G5
6,a100s+l102k+v105a+y168a+k172y-1,A1
7,a100s+l102k+v105a+y168a+k172y-3,C1
8,a100s+y168a+k172y-1,G4
9,a100s+y168a+k172y-3,A5
10,a100y+l102k+v105a+y168a+k172y-1,H2
11,a100y+l102k+v105a+y168a+k172y-3,B3
12,l102a+c157s+y168h+k172s+m203w-3,F4
13,l102a+c157s+y168h+m203w-1,A4
14,l102a+c157w+y168h+m203w-2,E1
15,l102a+c157w+y168h+m203w-3,F1
16,l102a+s155q+y168h+m203w-1,A7
17,l102a+s155q+y168h+m203w-2,B7
18,l102a+s155q+y168h+m203w-3,C7
19,l102a+v105h+s155a+y168h+a195c+m203w+a207m-1,C3
20,l102a+v105h+s155a+y168h+a195c+m203w+a207m-2,D3
21,l102a+v105h+s155a+y168h+a195c+m203w-3,E6
22,l102a+v105h+s155a+y168h+m203w-2,C2
23,l102a+v105h+s155q+y168h+m203w-1,F3
24,l102a+v105h+s155q+y168h+m203w-2,G3
25,l102a+y168h+m203w-2,H7
26,l102a+y168h+m203w-3,A8
27,l102g+v105g+y168a+k172y+n201d+q204a+a207h

In [ ]:
-